In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import torch
import torch.nn as nn
import gradio as gr
from ultralytics import YOLO
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os

# --- STEP 1: LOAD YOUR TRAINED CLASSIFIER ---
def load_pepper_model(model_path):
    model = models.resnet18(weights=None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found at: {model_path}")
        
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()
    return model

# --- STEP 2: SETUP ---
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# UPDATE THIS PATH to your actual .pth file location
MODEL_PATH = '/content/drive/MyDrive/pepper_resnet18_final.pth'

detector = YOLO('yolov8n.pt') 
classifier = load_pepper_model(MODEL_PATH)
class_names = ['Healthy', 'Unhealthy'] 

# --- STEP 3: LOGIC FOR GRADIO ---
def predict_pepper(input_img):
    # Gradio provides images as numpy arrays by default
    img = np.array(input_img)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_display = img_bgr.copy()
    
    results = detector(img)
    detections_found = False
    
    for result in results:
        if len(result.boxes) > 0:
            detections_found = True
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                process_and_draw(img_display, img, x1, y1, x2, y2)
    
    if not detections_found:
        h, w, _ = img.shape
        process_and_draw(img_display, img, 0, 0, w, h)

    # Return as RGB for Gradio to display correctly
    return cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)

def process_and_draw(img_display, img_rgb, x1, y1, x2, y2):
    crop = img_rgb[y1:y2, x1:x2]
    if crop.size == 0: return
    
    pil_img = Image.fromarray(crop)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    
    with torch.no_grad():
        output = classifier(input_tensor)
        index = torch.argmax(output, dim=1).item()
        label = class_names[index]
    
    color = (0, 255, 0) if label == 'Healthy' else (0, 0, 255) # BGR
    cv2.rectangle(img_display, (x1, y1), (x2, y2), color, 5)
    cv2.putText(img_display, label, (x1 + 5, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

# --- STEP 4: LAUNCH GRADIO ---
demo = gr.Interface(
    fn=predict_pepper,
    inputs=gr.Image(label="Drag & Drop Pepper Image here"),
    outputs=gr.Image(label="Analysis Result"),
    title="🌶️ PepperAI: Health Scanner",
    description="Upload an image of a pepper plant to detect health issues."
)

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d4f64437274c2a69c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x416 (no detections), 385.5ms
Speed: 31.0ms preprocess, 385.5ms inference, 25.3ms postprocess per image at shape (1, 3, 640, 416)

0: 512x640 1 banana, 207.5ms
Speed: 4.5ms preprocess, 207.5ms inference, 24.0ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 banana, 280.5ms
Speed: 6.0ms preprocess, 280.5ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 640)

0: 448x640 (no detections), 158.3ms
Speed: 5.2ms preprocess, 158.3ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 152.5ms
Speed: 4.7ms preprocess, 152.5ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)
Created dataset file at: .gradio/flagged/dataset1.csv

0: 640x480 (no detections), 185.4ms
Speed: 6.0ms preprocess, 185.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 163.8ms
Speed: 5.7ms preprocess, 163.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x640 